###This Colab notebook provides a method for **calculating Gaussian and mean curvatures** based on input data files representing 3D points.

####The points read from the user's files are the XYZ coordinates of an atom representing the membrane lipid head during molecular dynamics simulations.

####The user must provide XYZ files renamed as (file_base_name)_(sequential_number).xyz.
####The user specifies the file_base_name (default set to M1 based on the example files) in the first widget and the total number of files to analyze in the second widget (default set to 3 as we have provided three files).

####In the provided example files, we include three frames from a molecular dynamics simulation of a membrane composed of 398 lipids (water and salts were removed from the files). To identify the head of each lipid type, we saved the positions of the phosphorus atom (P) for POPC and sphingomyelin (OSM) and the nitrogen (N) atom for ceramide (CER181). The files are named M1_1.xyz, M1_2.xyz, and M1_3.xyz.


###This Google Colab notebook **is supplementary material for the paper "Bending the rules: molecular dynamics of hydroxylated sphingolipid membranes with 2-hydroxyoleic acid" (link here),** and we encourage users to read the paper before using the tool.

####If you use our tool, we kindly ask you to cite the article "link"




In [ ]:
!pip install plotly
!pip install ipywidgets
!pip install scipy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# @title TOTAL CALCULATION OF CURVATURES AND COEFFICIENTS (update 18/11/2024)
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from scipy.interpolate import griddata

def read_and_reorient_points(filepath):
    # Read data from file, skipping the first line which contains the number of points
    data = np.loadtxt(filepath, skiprows=1, usecols=(1, 2, 3))

    # Reassign columns: x remains x, y becomes z, z becomes y
    x = data[:, 0]
    y = data[:, 2]  # Originally z
    z = data[:, 1]  # Originally y

    # Distinguish surfaces based on the new z axis
    z_threshold = np.median(z)  # Use the median of z as the threshold for separation
    surface1 = data[z < z_threshold]
    surface2 = data[z >= z_threshold]

    return surface1, surface2

def create_grids(surface):
    x = surface[:, 0]
    y = surface[:, 2]  # Originally z
    z = surface[:, 1]  # Originally y

    # Create a grid of points
    grid_x, grid_y = np.mgrid[min(x):max(x):100j, min(y):max(y):100j]

    # Interpolate z-values on the grid using the ‘linear’ method
    grid_z = griddata((x, y), z, (grid_x, grid_y), method='linear')

    # Manage any NaN that might still form
    if np.any(np.isnan(grid_z)):
        grid_z = griddata((x, y), z, (grid_x, grid_y), method='nearest')

    return grid_x, grid_y, grid_z



def curvature_calculation(grid_x, grid_y, grid_z):
    # Calculate the spacing between points in the grid
    dx = grid_x[1, 0] - grid_x[0, 0]  # Spacing in x direction
    dy = grid_y[0, 1] - grid_y[0, 0]  # Spacing in y direction

    # Calculate the first derivatives
    dz_dx = np.gradient(grid_z, dx, axis=0)
    dz_dy = np.gradient(grid_z, dy, axis=1)

    # Calculate the second derivatives
    d2z_dxdx = np.gradient(dz_dx, dx, axis=0)
    d2z_dydy = np.gradient(dz_dy, dy, axis=1)
    d2z_dxdy = np.gradient(dz_dx, dy, axis=1)

    # Calculate the coefficients for the formulas
    E = 1 + dz_dx**2
    F = dz_dx * dz_dy
    G = 1 + dz_dy**2

    # Calculate Gaussian curvature K
    numerator_K = d2z_dxdx * d2z_dydy - d2z_dxdy**2
    denominator_K = (E * G - F**2)**2  # Corrected line

    K = numerator_K / denominator_K

    # Calculate mean curvature H
    numerator_H = (G * d2z_dxdx - 2 * F * d2z_dxdy + E * d2z_dydy)
    denominator_H = 2 * (E * G - F**2)**(1.5)

    H = numerator_H / denominator_H

    return K, H

def calculate_area(grid_x, grid_y, grid_z):
    # Obtaining grid dimensions
    nrows, ncols = grid_x.shape

    area_total = 0.0

    #Loop through the grid for each cell
    for i in range(nrows - 1):
        for j in range(ncols - 1):
            x0 = grid_x[i, j]
            y0 = grid_y[i, j]
            z0 = grid_z[i, j]

            x1 = grid_x[i+1, j]
            y1 = grid_y[i+1, j]
            z1 = grid_z[i+1, j]

            x2 = grid_x[i, j+1]
            y2 = grid_y[i, j+1]
            z2 = grid_z[i, j+1]

            x3 = grid_x[i+1, j+1]
            y3 = grid_y[i+1, j+1]
            z3 = grid_z[i+1, j+1]

            # First triangle (v0, v1, v2)
            area1 = area_triangle(x0, y0, z0, x1, y1, z1, x2, y2, z2)
            # Second triangle(v2, v1, v3)
            area2 = area_triangle(x2, y2, z2, x1, y1, z1, x3, y3, z3)

            # Sum of the areas of triangles
            area_total += area1 + area2

    return area_total

def area_triangle(x0, y0, z0, x1, y1, z1, x2, y2, z2):

    #  Vectors of the triangle sides
    v0 = np.array([x1 - x0, y1 - y0, z1 - z0])
    v1 = np.array([x2 - x0, y2 - y0, z2 - z0])

    # Cross product of the side vectors
    cross = np.cross(v0, v1)

    # Area of the triangle = half of the norm of the cross product
    area = 0.5 * np.linalg.norm(cross)

    return area


def main(base_name):
    A = []  # List to store surface areas
    H_media = []  # List to store average mean curvatures
    K_media = []  # List to store average Gaussian curvatures
    n_files = num_files.value  # Get the number of files to analyze from user input
    n_files = num_files.value  # Extract integer value from the IntText widget
    for i in range(1, n_files + 1):
        # Construct the file path for input data
        file_path = f'/content/drive/MyDrive/{base_name}_{i}.xyz'

        # Read and reorient points from the file to define two surfaces
        surface1, surface2 = read_and_reorient_points(file_path)

        # Create grids for each surface based on points
        grid_x1, grid_y1, grid_z1 = create_grids(surface1)
        grid_x2, grid_y2, grid_z2 = create_grids(surface2)

        # Calculate Gaussian and mean curvatures for each surface
        curvature_gaussiana1, curvature_media1 = curvature_calculation(grid_x1, grid_y1, grid_z1)
        curvature_gaussiana2, curvature_media2 = curvature_calculation(grid_x2, grid_y2, grid_z2)

        # Calculate areas for each surface
        area1 = calculate_area(grid_x1, grid_y1, grid_z1)
        area2 = calculate_area(grid_x2, grid_y2, grid_z2)

        # Store results
        A.append(area1)
        H_media.append(np.mean(curvature_media1))
        K_media.append(np.mean(curvature_gaussiana1))

        print(f" Processing file {base_name}_{i}")

    E_0 = 1  # Constant energy value

    # Construct the vector b with inverse area terms
    b = np.array([E_0 / A_i for A_i in A])

    # Construct the matrix X for the linear system
    X = np.array([[2 * H_i**2, K_i, 1] for H_i, K_i in zip(H_media, K_media)])

    # Solve the linear system using the least squares method
    p, residuals, rank, s = np.linalg.lstsq(X, b, rcond=None)

    # Extract the values of kappa, kappa_G, and gamma
    kappa, kappa_G, gamma = p

    return A, H_media, K_media, kappa, kappa_G, gamma

file_base_name = widgets.Text(
    value='M1',
    description='File Base Name:',
    disabled=False,
    layout=widgets.Layout(width='200px')   # Adjust the width as needed
)

# Widget to input the number of files to analyze
num_files = widgets.IntText(
    value=3,  # Default value
    description='Num Files:',
    disabled=False,
    layout=widgets.Layout(width='200px')  # Adjust the width as needed
)

# Button to execute the function
execute_button = widgets.Button(
    description='Run',
    disabled=False,
    button_style='success'
)

# Output widget for displaying results
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        base_name = file_base_name.value
        A, H_media, K_media, kappa, kappa_G, gamma = main(base_name)

        print(f"kappa: {kappa}, kappa_G: {kappa_G}, gamma: {gamma}")


# Connect button click event
execute_button.on_click(on_button_click)

# Display widgets
display(file_base_name, num_files, execute_button, output)






Text(value='M1', description='File Base Name:', layout=Layout(width='200px'))

IntText(value=3, description='Num Files:', layout=Layout(width='200px'))

Button(button_style='success', description='Run', style=ButtonStyle())

Output()